In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import openai
import os
from dotenv import load_dotenv
from openai import OpenAI
from typing import List, Dict, Any
import json
from PIL import Image, ImageDraw, ImageFont
import base64
import io
from paddleocr import PaddleOCR, draw_ocr
import numpy as np

# 환경변수 로드 및 GPT 클라이언트 초기화
load_dotenv(dotenv_path=".env")

/opt/anaconda3/envs/choi_env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


True

In [2]:
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# 디버깅용 출력
print("🔑 API KEY 불러오기 상태:", "성공" if api_key else "실패")
print("🔑 API KEY 일부:", api_key[:10] if api_key else "None")

if not api_key:
    raise ValueError("❌ 환경변수 OPENAI_API_KEY를 찾을 수 없습니다.")

🔑 API KEY 불러오기 상태: 성공
🔑 API KEY 일부: sk-proj-ae


In [4]:
import base64
import json
import requests
image_path = "/workspace/ppocr_test/we.jpg"
# 1. 이미지 파일을 base64로 인코딩
with open(image_path, "rb") as img_file:
    img_base64 = base64.b64encode(img_file.read()).decode("utf-8")

# 2. 요청 데이터 구성
payload = {
    "user_prompt": "부산에서 출발하고 대전으로 도착하는 내일 아침 9시에 성인 3명 기차 예약해줘",
    "image_base64": img_base64
}

# 3. POST 요청 전송
response = requests.post("http://127.0.0.1:8000/analyze", json=payload)

# 4. 결과 출력
print(response.json())


{'selected_boxes': [{'text': '서울', 'box': [[648.0, 940.0], [747.0, 940.0], [747.0, 1010.0], [648.0, 1010.0]]}, {'text': '부산', 'box': [[648.0, 1231.0], [753.0, 1231.0], [753.0, 1301.0], [648.0, 1301.0]]}, {'text': '2025 03 25 오전 10:00', 'box': [[348.0, 1545.0], [933.0, 1545.0], [933.0, 1604.0], [348.0, 1604.0]]}, {'text': '총 1명', 'box': [[627.0, 1842.0], [768.0, 1842.0], [768.0, 1909.0], [627.0, 1909.0]]}, {'text': '열차 조회', 'box': [[618.0, 2203.0], [816.0, 2203.0], [816.0, 2271.0], [618.0, 2271.0]]}]}


In [33]:

# 이미지 열기
image = Image.open(image_path).convert("RGB")
draw = ImageDraw.Draw(image)

# 글꼴 (선택 사항)
try:
    font = ImageFont.truetype("/workspace/MALGUN.TTF", 24)
except:
    font = ImageFont.load_default()

result = response.json()
# 박스와 텍스트 시각화
for item in result["selected_boxes"]:
    box = [tuple(point) for point in item["box"]]  # 리스트 → 튜플
    draw.polygon(box, outline="red", width=3)
    draw.text((box[0][0], box[0][1] - 25), item["text"], fill="red", font=font)

# 결과 저장
output_path = "/workspace/output/selected_boxes_viz.jpg"
image.save(output_path)
print(f"✅ 박스 시각화 완료: {output_path}")

✅ 박스 시각화 완료: /workspace/output/selected_boxes_viz.jpg


## 여러테스트

In [5]:
import base64
import json
import requests
image_path = "/workspace/ppocr_test/we.jpg"
# 1. 이미지 파일을 base64로 인코딩
with open(image_path, "rb") as img_file:
    img_base64 = base64.b64encode(img_file.read()).decode("utf-8")

# 2. 요청 데이터 구성
payload = {
    "user_prompt": "서울에서 출발해서 광주까지 가는 기차표 내일 오전 9시로 성인 셋 예약해줘",
    "image_base64": img_base64
}

# 3. POST 요청 전송
response = requests.post("http://127.0.0.1:8000/analyze", json=payload)

# 4. 결과 출력
print(response.json())


# 이미지 열기
image = Image.open(image_path).convert("RGB")
draw = ImageDraw.Draw(image)

# 글꼴 (선택 사항)
try:
    font = ImageFont.truetype("/workspace/MALGUN.TTF", 24)
except:
    font = ImageFont.load_default()


result = response.json()

# 박스와 텍스트 시각화
for item in result["selected_boxes"]:
    box = [tuple(point) for point in item["box"]]  # 리스트 → 튜플
    draw.polygon(box, outline="red", width=3)
    draw.text((box[0][0], box[0][1] - 25), item["text"], fill="red", font=font)

# 결과 저장
output_path = "/workspace/output/selected_boxes1.jpg"
image.save(output_path)
print(f"✅ 박스 시각화 완료: {output_path}")

{'selected_boxes': [{'text': '서울', 'box': [[648.0, 940.0], [747.0, 940.0], [747.0, 1010.0], [648.0, 1010.0]]}, {'text': '부산', 'box': [[648.0, 1231.0], [753.0, 1231.0], [753.0, 1301.0], [648.0, 1301.0]]}, {'text': '2025 03 25 오전 10:00', 'box': [[348.0, 1545.0], [933.0, 1545.0], [933.0, 1604.0], [348.0, 1604.0]]}, {'text': '총 1명', 'box': [[627.0, 1842.0], [768.0, 1842.0], [768.0, 1909.0], [627.0, 1909.0]]}, {'text': '열차 조회', 'box': [[618.0, 2203.0], [816.0, 2203.0], [816.0, 2271.0], [618.0, 2271.0]]}]}
✅ 박스 시각화 완료: /workspace/output/selected_boxes1.jpg


✅ 박스 시각화 완료: /workspace/output/selected_boxes.jpg
